# Gathering MRI Data

In [5]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip

In [6]:
proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
template_folder =  os.path.join(proj_root, "templates/")
output_folder = os.path.join(proj_root, "output/")

project root: /Users/joachimpfefferkorn/repos/neurovolume


In [7]:
raw_anat_img = ants.image_read(t1_input_filepath)

In [8]:
print(raw_anat_img)
explore_3D_vol(raw_anat_img.numpy(), dim="x")
explore_3D_vol(raw_anat_img.numpy(), dim="y")
explore_3D_vol(raw_anat_img.numpy(), dim="z")

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 296)
	 Spacing    : (0.4785, 0.4785, 0.5)
	 Origin     : (119.989, 104.52, -84.2457)
	 Direction  : [-1.      0.0025  0.     -0.0025 -1.      0.      0.      0.      1.    ]



interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=147, description='slice', max=295), Output()), _dom_classes=('widget-int…

# Skull Stripping
Using ANTs

[this notebook](https://github.com/Angeluz-07/MRI-preprocessing-techniques/blob/main/notebooks/09_brain_extraction_with_template.ipynb) is what we're basing this on

## Load Template and Mask

In [9]:
mni_template_img = ants.image_read(f"{template_folder}/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_template_mask = ants.image_read(f"{template_folder}/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")

In [10]:
print(mni_template_img)
explore_3D_vol(mni_template_img.numpy())

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (197, 233, 189)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, 134.0, -72.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]



interactive(children=(IntSlider(value=98, description='slice', max=196), Output()), _dom_classes=('widget-inte…

In [8]:
print(mni_template_mask)
explore_3D_vol(mni_template_mask.numpy())

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (197, 233, 189)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, 134.0, -72.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]



interactive(children=(IntSlider(value=98, description='slice', max=196), Output()), _dom_classes=('widget-inte…

## Register Template and Masks to Raw image
Here we warp the MNI template

In [9]:
template_warp_to_raw_anat = ants.registration(
    fixed=raw_anat_img,
    moving=mni_template_img, 
    type_of_transform='SyN',
    verbose=True
)

antsRegistration -d 3 -r [0x30006aea8,0x17f92f148,1] -m mattes[0x30006aea8,0x17f92f148,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[0x30006aea8,0x17f92f148,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpi3ysy5jm,0x30006be88,0x30006b628] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1
All_Command_lines_OK
Using single precision for computations.
The composite transform comprises the following transforms (in order): 
  1. Center of mass alignment using fixed image: 0x30006aea8 and moving image: 0x17f92f148 (type = Euler3DTransform)
  Reading mask(s).
    Registration stage 0
      No fixed mask
      No moving mask
    Registration stage 1
      No fixed mask
      No moving mask
  number of levels = 4
  number of levels = 3
  fixed image: 0x30006aea8
  moving image: 0x17f92f148
  fixed image: 0x30006aea8
  moving image: 0x17f92f148

KeyboardInterrupt: 

In [10]:
registered_template_img = template_warp_to_raw_anat['warpedmovout']

The following comparison function asserts that the new `registered_template_img` image has been transformed to the size and shape of the `raw_anat_img`

Next, we apply this to the mask so that it fits over the brain in the data we wish to visualize

In [11]:
brain_mask = ants.apply_transforms(
    fixed=template_warp_to_raw_anat['warpedmovout'],
    moving=mni_template_mask,
    transformlist=template_warp_to_raw_anat['fwdtransforms'],
    interpolator='nearestNeighbor',
    verbose=True
)

['-d', '3', '-i', '0x17f8ce6a8', '-o', '0x17f8ceb28', '-r', '0x17f8ce548', '-n', 'nearestNeighbor', '-t', '/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpucm2vqlj1Warp.nii.gz', '-t', '/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpucm2vqlj0GenericAffine.mat']
Using double precision for computations.
Input scalar image: 0x17f8ce6a8
Could not create ImageIO for the input file, assuming dimension = 3 and scalar pixel type
Reference image: 0x17f8ce548
The composite transform comprises the following transforms (in order): 
  1. /var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpucm2vqlj0GenericAffine.mat (type = AffineTransform)
  2. /var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpucm2vqlj1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: NearestNeighborInterpolateImageFunction
Output warped image: 0x17f8ceb28


In [ ]:
#explore(raw_anat_img.numpy(), brain_mask.numpy())
explore_3D_vol(raw_anat_img.numpy(), mask=brain_mask.numpy())

interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…

Let's fine tune the mask a bit

In [13]:
brain_mask_dilated = ants.morphology(brain_mask, radius=4, operation='dilate', mtype='binary')

In [14]:
explore_3D_vol(raw_anat_img.numpy(), mask=brain_mask_dilated.numpy())

interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…

# Mask the Image and Create Sections

In [15]:
brain_anat = ants.mask_image(raw_anat_img, brain_mask_dilated)

In [16]:
print(brain_anat)
explore_3D_vol(brain_anat.numpy())

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 296)
	 Spacing    : (0.4785, 0.4785, 0.5)
	 Origin     : (119.989, 104.52, -84.2457)
	 Direction  : [-1.      0.0025  0.     -0.0025 -1.      0.      0.      0.      1.    ]



interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…

# Building Scivol

Initialize the scivol object

In [17]:
skullstrip = Scivol("skullstrip")

Create the volumes:

In [18]:
full_anat_vol = create_normalized_volume(raw_anat_img.numpy())
brain_anat_vol = create_normalized_volume(brain_anat.numpy())

Create the grids:

In [19]:
explore_3D_array(full_anat_vol)
explore_3D_array(brain_anat_vol)

interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

In [20]:
skullstrip.add_grids([Grid("full_anat",[full_anat_vol]), Grid("brain_anat", [brain_anat_vol])])

writing full_anat to grid
writing brain_anat to grid


In [21]:
print(skullstrip)
skullstrip.save_scivol(output_folder)

skullstrip
tolerance: 0.0
affine: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
grids:
    - full_anat
    - brain_anat

writing skullstrip scivol dictionary
Encoding JSON file to utf-8
saving skullstrip.scivol to /Users/joachimpfefferkorn/repos/neurovolume/output/


Lets' quickly test this in our notebook

In [22]:
test_compressed_file_path = "/Users/joachimpfefferkorn/repos/neurovolume/output/skullstrip.scivol"
with gzip.open(test_compressed_file_path, 'r') as svin:
    scivol_bytes = svin.read()
scivol_str = scivol_bytes.decode('utf-8')

In [23]:
scivol_dict = json.loads(scivol_str)

In [26]:
print(type(scivol_dict))
print(scivol_dict['name'])
print([grid for grid in scivol_dict['grids']])
brain_anat_test_vol = np.asanyarray(scivol_dict['grids']['brain_anat']['frames'][0])
print(type(brain_anat_test_vol))
explore_3D_vol(brain_anat_test_vol)

<class 'dict'>
skullstrip
['full_anat', 'brain_anat']
<class 'numpy.ndarray'>


interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…

In [ ]:
explore_3D_vol(np.asanyarray(scivol_dict['grids']['full_anat']['frames'][0]))

interactive(children=(IntSlider(value=255, description='slice', max=511), Output()), _dom_classes=('widget-int…